# Task definition
Implement LSTM Sentiment Tagger for imdb reviews dataset.

1. (5pt) Fill missing code below
    * 1pt implement vectorization
    * 2pt implement \_\_init\_\_ and forward methods of models
    * 2pt implement collate function
2. (4pt) Implement training loop, choose proper loss function, use clear ml for max points.
    * 2pts is a baseline for well written, working code
    * 2pts if clear ml used properly
3. (3pt) Train the models (find proper hyperparams). Make sure you are not overfitting or underfitting. Visualize training of your best model (plot training, and test loss/accuracy in time). Your model should reach at least 87% accuracy. For max points it should exceed 89%. 
    * 1pt for accuracy above 89%
    * 1pt for accuracy above 87%
    * 1pt for visualizations

Remarks:
* Use embeddings of size 50
* Use 0.5 threshold when computing accuracy.
* Use supplied dataset for training and evaluation.
* You do not have to use validation set.
* You should monitor overfitting during training.
* For max points use clear ml to store and manage logs from your experiments. 
* We encourage to use pytorch lightning library (Addtional point for using it - however the sum must not exceed 12)

[Clear ML documentation](https://clear.ml/docs/latest/docs/)

[Clear ML notebook exercise from bootcamp](https://colab.research.google.com/drive/1wtLb4gg8beLS7smcyJlOZppn6_rQvSxL?usp=sharing)

In [1]:
device = 'cuda'

In [2]:
!pip install clearml

import os
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torchtext
from clearml import Task

import torch
from torch import nn
from torch import optim

from torch.utils.data import Dataset, DataLoader

In [91]:

web_server = 'https://app.community.clear.ml'
api_server = 'https://api.community.clear.ml'
files_server = 'https://files.community.clear.ml'
access_key = ''#@param {type:"string"}
secret_key = ''#@param {type:"string"}

Task.set_credentials(web_host=web_server,
                     api_host=api_server,
                     files_host=files_server,
                     key=access_key,
                     secret=secret_key)

In [4]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1hK-3iiRPlbePb99Fe-34LJNZ5yB-nduq
!tar -xvzf imdb_dataset.gz
data = pd.read_csv("imdb_dataset.csv")

In [6]:
from collections import Counter
from keras.preprocessing.sequence import pad_sequences
import re

In [8]:
from nltk.corpus import stopwords

In [11]:
class NaiveVectorizer:
    def __init__(self, tokenized_data, **kwargs):
        """Converts data from string to vector of ints that represent words. 
        Prepare lookup dict (self.wv) that maps token to int. Reserve index 0 for padding.
        """
        tokenized_data = self._clean_text(tokenized_data)
        tokenized_data = [seq.split() for seq in tokenized_data]
        self.wv = self._init_wv(tokenized_data)
        
    
    def _clean_text(self, data):
        """Remove special signs, leaving only letters, digits and spaces."""
        return [re.sub('[^A-Za-z0-9\s]+', '', seq) for seq in data]

    def _init_wv(self, tokenized_data):
        """Initialize token dictionary."""
        tokenized_data = [item for sublist in tokenized_data for item in sublist]
        tokenized_data = set(tokenized_data)
        non_stopwords = list(set(tokenized_data).difference(set(stopwords.words("english"))))
        return {token: i + 1 for i, token in enumerate(non_stopwords)}


    def vectorize(self, tokenized_seq):
        """Converts sequence of tokens into sequence of indices.
        If the token does not appear in the vocabulary(self.wv) it is ommited
        Returns torch tensor of shape (seq_len,) and type long."""
        tokenized_seq = self._clean_text(tokenized_seq)
        converted_seq = [self.wv[token] for token in tokenized_seq if token in self.wv.keys()]
        return torch.tensor(converted_seq)


class ImdbDataset(Dataset):
    SPLIT_TYPES = ["train", "test", "unsup"]

    def __init__(self, data, preprocess_fn, split="train"):
        super(ImdbDataset, self).__init__()
        if split not in self.SPLIT_TYPES:
            raise AttributeError(f"No such split type: {split}")

        self.split = split
        self.label = [i for i, c in enumerate(data.columns) if c == "sentiment"][0]
        self.data_col = [i for i, c in enumerate(data.columns) if c == "tokenized"][0]
        self.data = data[data["split"] == self.split]
        self.preprocess_fn = preprocess_fn

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.preprocess_fn(self.data.iloc[idx, self.data_col].split())
        label = self.data.iloc[idx, self.label]
        return (seq, label)


naive_vectorizer = NaiveVectorizer(data.loc[data["split"] == "train", "tokenized"])


def get_datasets():
    train_dataset = ImdbDataset(data, naive_vectorizer.vectorize)
    test_dataset = ImdbDataset(data, naive_vectorizer.vectorize, split="test")
        
    return train_dataset, test_dataset


def custom_collate_fn(pairs):
    """This function is supposed to be used by dataloader to prepare batches
    Input: list of tuples (sequence, label)
    Output: sequences_padded_to_the_same_lenths, original_lenghts_of_sequences, lables.
    torch.nn.utils.rnn.pad_sequence might be usefull here
    """
    seqcs = [pair[0] for pair in pairs]
    labels = [pair[1] for pair in pairs]
    lengths = torch.tensor([len(seq) for seq in seqcs])
    seqcs = torch.nn.utils.rnn.pad_sequence(seqcs, batch_first=True)
    
    return seqcs, lengths, torch.tensor(labels).float()

In [118]:
import torch.nn.functional as F

In [64]:
"""Implement LSTMSentimentTagger. 
The model should use a LSTM module.
Use torch.nn.utils.rnn.pack_padded_sequence to optimize processing of sequences.
When computing vocab_size of embedding layer remeber that padding_symbol counts to the vocab.
Use sigmoid activation function.
"""
class LSTMSentimentTagger(torch.nn.Module):
    def __init__(self, vocab_dim, hidden_dim, batch_size, num_layers=1, device='cuda'):
        super(LSTMSentimentTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        
        self.embed = torch.nn.Embedding(vocab_dim + 1, 50, padding_idx=0)
        self.layernorm1 = torch.nn.LayerNorm(50)

        self.lstm = torch.nn.LSTM(
            50, self.hidden_dim, num_layers, batch_first=True, bidirectional=False, dropout=0.2
        )
        self.hidden = self.init_hidden(batch_size, num_layers, device)
        self.layernorm2 = torch.nn.LayerNorm(self.hidden_dim * num_layers)
        
        self.dropout = torch.nn.Dropout(0.25)
        
        self.fc1 = torch.nn.Linear(self.hidden_dim * num_layers, 1)

    def init_hidden(self, batch_size, num_layers, device='cuda'):
        return (torch.autograd.Variable(torch.zeros(num_layers, batch_size, self.hidden_dim).to(device)),
                torch.autograd.Variable(torch.zeros(num_layers, batch_size, self.hidden_dim).to(device)))

    def forward(self, x, lengths):
        x = self.embed(x)
        x = self.dropout(x)
        x = self.layernorm1(x)
        x = torch.nn.utils.rnn.pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        _, (hn, _) = self.lstm(x, self.hidden)
        x = hn.permute(1, 0, 2)
        x = x.reshape(self.batch_size, -1)
        x = self.layernorm2(x)

        x = self.dropout(x)
        x = self.fc1(x)
        return torch.sigmoid(x)

# Trainig loop and visualizations


In [110]:
def train(model, train_loader, optimizer, loss_f, device='cuda'):
    model.train()
    train_loss = []
    correct = 0
    
    for x in train_loader:
        data, lengths, target = x
        data, lengths, target = data.to(device), lengths.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data, lengths).flatten().float()
        pred = torch.tensor([1 if pred_i >= 0.5 else 0 for pred_i in output]).to(device)
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss = loss_f(output, target)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    
    return np.array([
        np.mean(train_loss), correct / len(train_loader.dataset)
    ])

In [111]:
def test(model, test_loader, loss_f, device='cuda'):
    model.eval()
    test_loss = []
    correct = 0

    with torch.no_grad():
        for x in test_loaders:
            data, lengths, target = x
            data, lengths, target = data.to(device), lengths.to(device), target.to(device)
            output = model(data, lengths).flatten().float()
            test_loss.append(loss_f(output, target).item())
            pred = torch.tensor([1 if pred_i >= 0.5 else 0 for pred_i in output]).to(device)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = np.mean(test_loss)
    return np.array([test_loss, correct / len(test_loader.dataset)])

In [119]:
train_dataset, test_dataset = get_datasets()

In [123]:
config = {
    'seed': 256,
    'n_epochs': 20,
    'lr': 1e-3,
    'patience': 3,
    'batch_size': 500
}

In [124]:
train_loader = DataLoader(
    dataset=train_dataset, batch_size=config['batch_size'], shuffle=True, 
    collate_fn=custom_collate_fn,
)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=config['batch_size'], shuffle=False, 
    collate_fn=custom_collate_fn,
)

In [125]:
from clearml import Task


task = Task.create(project_name='gsn', task_name='lstm_tagger')
task.mark_started()
logger = task.get_logger()

In [126]:
torch.manual_seed(config['seed'])
model = LSTMSentimentTagger(len(naive_vectorizer.wv), 32, config['batch_size'], num_layers=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=config['patience'])
loss_f = torch.nn.BCELoss().to(device)

In [127]:
for i in range(config['n_epochs']):
    train_loss, train_acc = train(model, train_loader, optimizer, loss_f)
    test_loss, test_acc = test(model, test_loader, loss_f)
    print((f'{i:3} | train loss: {train_loss:.5f}, train_acc: {train_acc:.3f}, '
           f'test loss: {test_loss:.5f} test_acc: {test_acc:.3f}'))
    
    logger.report_scalar(title='Loss', series='Train', iteration=i, value=train_loss)
    logger.report_scalar(title='Loss', series='Test', iteration=i, value=test_loss)
    logger.report_scalar(title='Accuracy', series='Train', iteration=i, value=train_acc)
    logger.report_scalar(title='Accuracy', series='Test', iteration=i, value=test_acc)

    if scheduler:
        scheduler.step(test_loss)


task.mark_completed()
task.close()

## Visualizations

In [128]:
import pandas as pd
import plotly.express as px

In [135]:
task = Task.get_tasks(
    project_name='gsn',
    tags=['best']
)[0]

metrics = task.get_reported_scalars()

fig = px.line(pd.DataFrame({'train': metrics['Accuracy']['Train']['y'], 
                            'test': metrics['Accuracy']['Test']['y']}),
              labels={'index': 'Epochs',
                      'value': 'Accuracy',
                      'variable': 'Dataset'},
              title='Accuracy')
fig.show()

In [136]:
fig = px.line(pd.DataFrame({'train': metrics['Loss']['Train']['y'], 
                            'test': metrics['Loss']['Test']['y']}),
              labels={'index': 'Epochs',
                      'value': 'Loss',
                      'variable': 'Dataset'},
              title='Loss')
fig.show()